In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:46389")
client

<Client: 'tcp://127.0.0.1:46389' processes=8 threads=32, memory=125.87 GiB>

In [2]:
import numpy as np
import xarray as xr
from xmitgcm import open_mdsdataset
from xgcm.grid import Grid
import os.path as op
from fastjmd95 import rho as densjmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/tank/chaocean/qjamet/RUNS/ORAR/reruns2/'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'
savedir = '/tank/topog/tuchida/Wavelet-spectrum/'
tdgs = '/tank/tdgs/tuchida/TWA/'
topog = '/tank/topog/tuchida/TWA/'

In [4]:
def sound_speed(s,th,p):
    """
    s            : salinity (psu)
    th           : potential temperature (deg C, ITS-90)
    p            : pressure (dbar)
    """
    th2 = th**2
    sqrts = np.sqrt(s)

    anum = ( 9.99843699e2 +
            th *( 7.35212840 +
                 th *(-5.45928211e-2 +
                      th * 3.98476704e-4
                     )
                ) +
            s *( 2.96938239 -
                th * 7.23268813e-3 +
                s * 2.12382341e-3
               )
           )
    aden = (1. +
            th *( 7.28606739e-3 +
                 th *(-4.60835542e-5 +
                      th *( 3.68390573e-7 +
                           th * 1.80809186e-10
                          )
                     )
                ) +
            s *( 2.14691708e-3 -
                th *( 9.27062484e-6 +
                     th2 * 1.78343643e-10
                    ) +
                sqrts *( 4.76534122e-6 +
                        th2 * 1.63410736e-9
                       )
               )
           )
    

    anum_p = ( 1.04004591e-2 +
              th2 * 1.03970529e-7 +
              s * 5.18761880e-6
             )
    # aden_p = 5.30848875e-6 * xr.ones_like(s)
    aden_p = 5.30848875e-6 * np.ones_like(s)


    pth = p*th
    anum = anum + p *( 1.04004591e-2 +
                      th2 *  1.03970529e-7 +
                      s *  5.18761880e-6 -
                      p *( 3.24041825e-8 +
                          th2 *  1.23869360e-11
                         )
                     )
    aden = aden + p *( 5.30848875e-6 -
                      pth *(th2 *  3.03175128e-16 +
                            p *  1.27934137e-17
                           )
                     )


    anum_p = anum_p - p *( 6.4808365e-8 + th2 * 2.4773872e-11)

    aden_p = aden_p - pth *(th2 * 6.06350256e-16 +
                            p * 3.83802411e-17)


    rec_aden = aden**-1

    rho = anum*rec_aden
    rho_p = (anum_p - aden_p*rho)*rec_aden
    
    return np.sqrt(rho_p*1e-4)**-1

In [5]:
grav = 9.81
rho0 = 999.8
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
# Cp = 3.994e3
Cp = 3989.244953
# years = np.arange(2008,2013,dtype=int)
ySstart = -15
yNstart = 15
yend = 50
dnf = 2160
nfile = 73

In [6]:
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1419120,1576800,dnf).reshape(len(years),nfile)
dd = 0  # 
yystart = 0
ntimes[yystart,:]

array([1419120, 1421280, 1423440, 1425600, 1427760, 1429920, 1432080,
       1434240, 1436400, 1438560, 1440720, 1442880, 1445040, 1447200,
       1449360, 1451520, 1453680, 1455840, 1458000, 1460160, 1462320,
       1464480, 1466640, 1468800, 1470960, 1473120, 1475280, 1477440,
       1479600, 1481760, 1483920, 1486080, 1488240, 1490400, 1492560,
       1494720, 1496880, 1499040, 1501200, 1503360, 1505520, 1507680,
       1509840, 1512000, 1514160, 1516320, 1518480, 1520640, 1522800,
       1524960, 1527120, 1529280, 1531440, 1533600, 1535760, 1537920,
       1540080, 1542240, 1544400, 1546560, 1548720, 1550880, 1553040,
       1555200, 1557360, 1559520, 1561680, 1563840, 1566000, 1568160,
       1570320, 1572480, 1574640])

In [8]:
nmembs = np.arange(100,148)
xchunk = 100
ychunk = 100

yy = yystart
iiter = 1419120

for nmemb in nmembs:
    if nmemb == 100:
        ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['snap_diagOcn'], delta_t=2e2,
                             iters=iiter, 
                            ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,'XC':xchunk,'XG':xchunk})
    else:
        ds = xr.concat([ds, open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                                    % (nmemb)), 
                                            grid_dir=gdir,
                                            prefix=['snap_diagOcn'], delta_t=2e2,
                                            iters=iiter, 
                                           ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,'XC':xchunk,'XG':xchunk})
                       ], dim='nmemb')

            # ds = ds.chunk({'nmemb':6})
ds.coords['nmemb'] = ('nmemb',nmembs)

In [9]:
grid = Grid(ds, periodic=['X'])

pres = rho0*grav*(-ds.Z) * 1e-4
theta = ds.THETA.where(ds.maskC!=0.)
salt = ds.SALT.where(ds.maskC!=0.)

# Cs = sound_speed(salt, theta, pres).max(['nmemb','YC','XC'],skipna=True).compute()
Cs = xr.apply_ufunc(sound_speed, salt, theta, pres,
                    dask='parallelized', output_dtypes=[float,]
                   ).max(['nmemb','YC','XC'],skipna=True).compute()
Cs

<xarray.DataArray (Z: 46)>
array([1547.569 , 1547.6519, 1547.5643, 1547.0343, 1546.5657, 1545.2684,
       1545.0725, 1544.0138, 1543.9633, 1543.5817, 1543.8291, 1543.9569,
       1543.3118, 1542.5625, 1543.0859, 1543.691 , 1544.5132, 1545.455 ,
       1536.8256, 1533.8473, 1531.2787, 1528.2145, 1530.228 , 1532.4729,
       1534.9469, 1537.6427, 1540.5482, 1543.6482, 1546.9244, 1550.3579,
       1553.9292, 1557.62  , 1561.413 , 1565.2927, 1569.2455, 1573.2592,
       1577.3239, 1581.4305, 1585.5724, 1589.7432, 1593.9375, 1598.1519,
       1602.3824, 1606.6265, 1610.8816, 1615.1464], dtype=float32)
Coordinates:
    iter     int64 1419120
    time     timedelta64[ns] 3285 days
  * Z        (Z) >f4 -3.047 -9.454 -16.36 ... -5.375e+03 -5.625e+03 -5.875e+03
    drF      (Z) >f4 6.094 6.721 7.099 7.971 8.65 ... 248.9 249.6 249.9 250.5
    PHrefC   (Z) >f4 29.89 92.74 160.5 234.4 ... 5.273e+04 5.518e+04 5.764e+04
    rhoRef   (Z) >f4 1.021e+03 1.021e+03 1.021e+03 ... 1.044e+03 1.044e+03

In [10]:
dsave = Cs.to_dataset(name='Cs')
dsave.Cs.encoding = {}
dsave.to_netcdf(op.join(savedir,"Max-sound-speed_%010d.nc" % iiter), mode='w')

In [9]:
nmembs = np.arange(100,148)
xchunk = 100
ychunk = 100

yy = yystart

for itime in ntimes[yystart,:]:
    for nmemb in nmembs:
        if nmemb == 100:
            ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                         % (nmemb)), 
                                 grid_dir=gdir,
                                 prefix=['snap_diagOcn'], delta_t=2e2,
                                 iters=itime, 
                                ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,
                                                      'XC':xchunk,'XG':xchunk})
        else:
            ds = xr.concat([ds, open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                                        % (nmemb)), 
                                                grid_dir=gdir,
                                                prefix=['snap_diagOcn'], delta_t=2e2,
                                                iters=itime, 
                                               ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,
                                                                     'XC':xchunk,'XG':xchunk})
                           ], dim='nmemb')

                # ds = ds.chunk({'nmemb':6})
    ds.coords['nmemb'] = ('nmemb',nmembs)
    
    grid = Grid(ds, periodic=['X'])

    pres = rho0*grav*(-ds.Z) * 1e-4
    theta = ds.THETA.where(ds.maskC!=0.)
    salt = ds.SALT.where(ds.maskC!=0.)

    if itime == ntimes[0,0]:
        # Cs = sound_speed(salt, theta, pres).max(['nmemb','YC','XC'],skipna=True).compute()
        Cs = xr.apply_ufunc(sound_speed, salt, theta, pres,
                            dask='parallelized', output_dtypes=[float,]
                           ).max(['nmemb','YC','XC'],skipna=True).compute()
    else:
        Cs = np.maximum(Cs, xr.apply_ufunc(sound_speed, salt, theta, pres,
                            dask='parallelized', output_dtypes=[float,]
                           ).max(['nmemb','YC','XC'],skipna=True).compute())
    print(itime)
        
dsave = Cs.to_dataset(name='Cs')
dsave.Cs.encoding = {}
dsave.to_netcdf(op.join(savedir,"Max-snap-sound-speed_1967.nc" % itime), mode='w')

1419120
1421280
1423440
1425600
1427760
1429920
1432080
1434240
1436400
1438560
1440720
1442880
1445040
1447200
1449360
1451520
1453680
1455840
1458000
1460160
1462320
1464480
1466640
1468800
1470960
1473120
1475280
1477440
1479600
1481760
1483920
1486080
1488240
1490400
1492560
1494720
1496880
1499040
1501200
1503360
1505520
1507680
1509840
1512000
1514160
1516320
1518480
1520640
1522800
1524960
1527120
1529280
1531440
1533600
1535760
1537920
1540080
1542240
1544400
1546560
1548720
1550880
1553040
1555200
1557360
1559520
1561680
1563840
1566000
1568160
1570320
1572480
1574640
